In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
!pip install dython
from dython.nominal import compute_associations, associations
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_path = '/kaggle/input/dont-overfit-ii/train.csv'
test_path = '/kaggle/input/dont-overfit-ii/test.csv'
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [ ]:
df_train.head()

In [ ]:
df_train

In [ ]:
df_train.var()

In [ ]:
df_train['0']

In [ ]:
df_train['target'].value_counts()

In [ ]:
for i in range(0, 291, 10):
    ten = [str(x) for x in range(i, i+10)]
    ten.append('target')
    associations(df_train[ten], figsize=(10, 10))

In [ ]:
def draw_correlation_heat_map(data, col_list, label):
    plt.subplots(figsize=(15,2))
    corr_table = compute_associations(data[col_list])
#     print(corr_table[[label]].sort_values(by=[label]))
    sns.heatmap(corr_table[[label]].sort_values(by=[label]).T)
    plt.title(f"Fig. 3. Corelations between {label} and the rest of features.")
    return corr_table

In [ ]:
for i in range(0, 291, 10):
    ten = [str(x) for x in range(i, i+10)]
    ten.append('target')
    draw_correlation_heat_map(df_train, ten, 'target')

END

In [ ]:
max_corr_list = []
for i in range(0, 291, 10):
    ten = [str(x) for x in range(i, i+10)]
    ten.append('target')
    corr_table = compute_associations(df_train[ten])
    corr = corr_table[['target']].sort_values(by=['target']).T
    corr_index = corr[corr > 0.05].T.dropna().drop('target').index.tolist()
    print(corr_index)
    max_corr_list.extend(corr_index)

print(max_corr_list)

In [ ]:
df_train[max_corr_list].shape

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
# Apply PCA to the wine dataset
transformed_X = pca.fit_transform(df_train[max_corr_list])
# Look at the percentage of variance explained by the different components
print(pca.explained_variance_ratio_.round(3))

In [ ]:
def create_pca(df, pca):
    pca_names = ['pca_'+str(x) for x in range(10)]
    print(pca_names)
    pca_df = pd.DataFrame(pca.transform(df[max_corr_list])[:,0:10].reshape(-1,10), columns=pca_names)
    pca_df.info()
    return pca_df

In [ ]:
pca_df = create_pca(df_train, pca)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(pca_df, df_train['target'], test_size=0.2, stratify=df_train['target'])

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
y_train

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
rfc = Lasso(alpha=0.031, tol=0.01)
rfc.fit(x_train, y_train)

In [ ]:
y_hat_train = rfc.predict(x_train)
y_hat = rfc.predict(x_test)

In [ ]:
from sklearn.metrics import roc_auc_score
print('auc_train:', roc_auc_score(y_train, y_hat_train))
print('auc_test:', roc_auc_score(y_test, y_hat))

In [ ]:
rfc = Lasso(alpha=0.031, tol=0.01)
rfc.fit(pca_df, df_train['target'])

In [ ]:
X_test = create_pca(df_test, pca)

In [ ]:
y_hat = rfc.predict(X_test)

In [ ]:
df_test.head()

In [ ]:
sample_submission= pd.DataFrame({'id':df_test['id'].to_numpy(), 'target':y_hat })
sample_submission.to_csv(os.path.join('./',"submission.csv"), index=False)